In [1]:
import os
import pandas as pd
import plotly.express as px
import progressbar
import json
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
pd.set_option('display.max_colwidth', None)

In [2]:
data_path = '../src/data'
mapping_corpus_path = data_path + r'/product/lspc2020_to_tablecorpus'
mapping_corpus_path_2 = data_path + r'/product/lspc2020_to_tablecorpus/Cleaned'
table_corpus_path = data_path + r'/product/product_top100/cleaned'
table_corpus_path_with_id = data_path + r'/product/product_top100/cleaned/with_id'
table_corpus_path2 = data_path + r'/product/product_minimum3/cleaned/with_id'

In [3]:
df_joined_electronics = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_electronics_v2.json'), compression='gzip', orient='records', lines=True)

In [4]:
df_electronics_final = pd.read_excel("Final_Electronics_v3.xlsx")

In [9]:
df_electronics_clusters = df_joined_electronics[df_joined_electronics['cluster_id'].isin(df_electronics_final['cluster_id'].to_list())]

In [11]:
# 1100 for around 8000 entries and 267 distinct cluster_ids
df_electronics_clusters.drop_duplicates(subset=['table_id'])

,cluster_id,url,row_id,table_id,Valid,name,description,brand_x,brand_y
10402,823674,https://www.cartridgesave.co.uk/2970B010.html,2032,Product_cartridgesave.co.uk_September2020.json.gz,1,Genuine Black & Tri-Colour Canon PG-510 / CL-511 Ink Cartridge Multipack (2970B010),"Genuine Canon 510 & 511 Ink CartridgesContaining black and tri-colour ink, the Canon 2970B010 ink cartridge multipack contains everything you need to print high-quality documents. Produce stunning photographs at home with the convenient genuine Canon 510/511ink unit. The genuine Canon 2970B010 can deliver up to 220 pages of black ink and up to 244 pages of colour ink, enabling users to produce high-quality images on a wide range of media. The genuine Canon PG-510/CL-511 ink-jet cartridge can also be used in producing professional-looking documents. Graphics and texts delivered by the genuine Canon PG510/CL511 printer ink will be crystal clear and ultra-sharp. Quick Delivery On Genuine Canon PG-510/CL-511 Ink Cartridge At Cartridge SaveFind all of your printing needs here at Cartridge Save. Our sales team is dedicated in helping you find products such as the genuine Canon PG-510/CL-511 printer cartridges. If you're not sure if the genuine Canon 2970B010 cartridge unit will work on your unit, just check out our printer compatibility list. Cartridge Save accepts all major cards and don't worry about using your credit card to buy the genuine Canon 2970B010, we provide a secure sever where transactions take place. If you order the genuine Canon ink 510/511 today, you can get it the next day through our free delivery service. With award winning customer services on standby, feel free to use our phone or live chat services to ask a question about the genuine Canon 2970B010 cartridge.",Canon,canon
18006,323989,https://store.macs-4-u.com.au/usb-c-charge-cable-1-m.html,192,Product_macs-4-u.com.au_September2020.json.gz,1,USB-C Charge Cable (1 m),USB-C Charge Cable (1 m),None,None
20908,345616,https://www.scan.co.uk/products/6tb-wd-red-pro-wd6003ffbx-35-nas-hdd-sata-iii-6gb-s-7200rpm-256mb-cache-shock-sensor-ncq-oem,4945,Product_scan.co.uk_September2020.json.gz,1,"WD Red Pro 6TB 3.5\"" SATA NAS HDD/Hard Drive","6TB WD Red Pro WD6003FFBX, 3.5\"" NAS HDD, SATA III 6Gb/s, 7200rpm, 256MB Cache, Shock Sensor, NCQ, OEM",WD,None
25661,2374515,https://www.tecnoprecios.net/tinta-original/860-multipack-canon-pg-545-cl-546-blister-mg2450-mg2550.html,611,Product_tecnoprecios.net_September2020.json.gz,1,CARTUCHO TINTA CANON PG545 NEGRO + CL546 COLOR,MULTIPACK CANON PG-545/CL-546 BLISTER MG2450-MG2550,None,None
38388,3572453,https://co.catalog.lifemiles.com/p/nuevo-apple-pencil-2da-generacio-n-x9k7cv,4356,Product_lifemiles.com_September2020.json.gz,1,Nuevo Apple Pencil - 2da Generación,"Presentamos el nuevo Apple Pencil.El Apple Pencil reinventó la forma de dibujar, tomar notas y marcar documentos para hacerla más intuitiva, precisa y mágica. Y ahora el nuevo Apple Pencil lleva esa experiencia un paso más allá. Cambia de herramienta con sólo tocarlo dos veces, y conéctalo al dispositivo para enlazarlo y cargarlo de forma inalámbrica. Adelante, deja tu marca.Simplemente más poderoso.El nuevo Apple Pencil se conecta de forma magnética al costado del iPad Pro. Sólo tienes que acercarlo para que se acople con firmeza. Una vez conectado, se enlaza al dispositivo de forma automática y se empieza a cargar. El Apple Pencil también ofrece nuevas formas de usarlo. Con sólo tocarlo dos veces puedes cambiar de herramienta sin necesidad de soltarlo.Un nuevo diseño de una sola pieza. El nuevo Apple Pencil está diseñado para ser muy cómodo y fácil de usar. Como tiene un diseño simple, sin conectores ni piezas móviles, es muy fácil de guardar cuando lo terminas de usar. Incluso puedes hacerle un grabado para darle un toque personal.Se enlaza mágicamente. Y se carga inalámbricamente. El nuevo Apple Pencil empieza a cargarse apenas apoyas la superficie plana del lápiz sobre el costado derecho del iPad P

In [13]:
df_electronics_clusters.drop_duplicates(subset=['table_id'])['table_id']

10402           Product_cartridgesave.co.uk_September2020.json.gz
18006               Product_macs-4-u.com.au_September2020.json.gz
20908                    Product_scan.co.uk_September2020.json.gz
25661              Product_tecnoprecios.net_September2020.json.gz
38388                 Product_lifemiles.com_September2020.json.gz
                                    ...                          
24337282              Product_atbshop.co.uk_September2020.json.gz
24496127        Product_citybikesonline.com_September2020.json.gz
24499882              Product_blaines.co.uk_September2020.json.gz
24564082    Product_searshometownstores.com_September2020.json.gz
24643462         Product_justballgloves.com_September2020.json.gz
Name: table_id, Length: 1108, dtype: object

In [15]:
table_counts = df_electronics_clusters.groupby('table_id').count()

In [21]:
table_counts[table_counts ['cluster_id'].sort_values(ascending=False) > 15]

<ipython-input-21-a30fe86562f9>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  table_counts[table_counts ['cluster_id'].sort_values(ascending=False) > 15]


,cluster_id,url,row_id,Valid,name,description,brand_x,brand_y
table_id,,,,,,,,
Product_advantageappliance.com_September2020.json.gz,33,33,33,33,33,0,0,0
Product_aitoro.com_September2020.json.gz,43,43,43,43,43,0,0,0
Product_allstars.com.sg_September2020.json.gz,16,16,16,16,16,16,0,12
Product_appliancefactory.com_September2020.json.gz,37,37,37,37,37,0,0,0
Product_applianceliquidators.com_September2020.json.gz,59,59,59,59,59,0,0,0
...,...,...,...,...,...,...,...,...
Product_vohac.com_September2020.json.gz,68,68,68,68,68,0,0,0
Product_waadtappliance.com_September2020.json.gz,51,51,51,51,51,0,0,0
Product_whisknyc.com_September2020.json.gz,22,22,22,22,22,0,3,0


In [26]:
df_electronics_clusters[['cluster_id','row_id','table_id','name']]

,cluster_id,row_id,table_id,name
10402,823674,2032,Product_cartridgesave.co.uk_September2020.json.gz,Genuine Black & Tri-Colour Canon PG-510 / CL-511 Ink Cartridge Multipack (2970B010)
18006,323989,192,Product_macs-4-u.com.au_September2020.json.gz,USB-C Charge Cable (1 m)
20908,345616,4945,Product_scan.co.uk_September2020.json.gz,"WD Red Pro 6TB 3.5\"" SATA NAS HDD/Hard Drive"
25661,2374515,611,Product_tecnoprecios.net_September2020.json.gz,CARTUCHO TINTA CANON PG545 NEGRO + CL546 COLOR
38388,3572453,4356,Product_lifemiles.com_September2020.json.gz,Nuevo Apple Pencil - 2da Generación
...,...,...,...,...
24672648,238935,114,Product_shoptricountyfurniture.com_September2020.json.gz,25 cu. ft. Side-by-Side Refrigerator with In-Door Ice Maker in Black Stainless Steel
24674390,1107237,54,Product_carolinabrewsupply.com_September2020.json.gz,3 Way Gas Distributor - w/o PRV
24677077,3555009,86,Product_appliancegallerydayton.com_September2020.json.gz,26 cu. ft. Side-By-Side Refrigerator
24680049,17260023,67,Product_bandsappliance.com_September2020.json.gz,Linear Wash 39 dBA Dishwasher in Tuscan Stainless Steel


In [42]:
df_cluster_323989 = df_electronics_clusters[df_electronics_clusters['cluster_id']==323989][['cluster_id','row_id','table_id','name']]

In [28]:
def jaccard_similarity_score(original, translation):
    intersect = set(original).intersection(set(translation))
    union = set(original).union(set(translation))
    try:
        return len(intersect) / len(union)
    except ZeroDivisionError:
        return 0

In [29]:
def remove_punctuation(token_vector):
    return token_vector.apply(lambda token_list: [word for word in token_list if word not in string.punctuation])

In [30]:
def remove_stopwords(token_vector, stopwords_list):
    return token_vector.apply(lambda token_list: [word for word in token_list if word not in stopwords_list])

In [47]:
df_cluster_323989['product_tokens'] = df_cluster_323989['name'].apply(lambda row: row.lower()).apply(lambda row: word_tokenize(row))
df_cluster_323989['product_tokens'] = remove_stopwords(df_cluster_323989['product_tokens'],stopwords.words())
df_cluster_323989['product_tokens'] = remove_punctuation (df_cluster_323989['product_tokens'])
df_cluster_323989.reset_index(inplace=True)

In [48]:
df_cluster_323989

,index,cluster_id,row_id,table_id,name,product_tokens
0,18006,323989,192,Product_macs-4-u.com.au_September2020.json.gz,USB-C Charge Cable (1 m),"[usb-c, charge, cable, 1]"
1,1318608,323989,1064,Product_retravision.com.au_September2020.json.gz,Apple USB-C Charge Cable (1M),"[apple, usb-c, charge, cable, 1m]"
2,2826897,323989,152,Product_pskmegastore.com_September2020.json.gz,Apple MUF72ZM/A cavo USB 1 m USB C Bianco,"[apple, muf72zm/a, cavo, usb, 1, usb, bianco]"
3,3207585,323989,9,Product_ilovecomputers.com.au_September2020.json.gz,Apple Apple USB-C Charge Cable (1 m),"[apple, apple, usb-c, charge, cable, 1]"
4,12580514,323989,146,Product_mcarerental.com_September2020.json.gz,Apple USB-C Charge Cable 1m,"[apple, usb-c, charge, cable, 1m]"
5,15121729,323989,321,Product_alza.co.uk_September2020.json.gz,Apple USB-C Charging Cable 1m,"[apple, usb-c, charging, cable, 1m]"
6,15290618,323989,20,Product_lecbroome.com.au_September2020.json.gz,USB-C Charge Cable (2M),"[usb-c, charge, cable, 2m]"
7,16279424,323989,8,Product_expotronica.com_September2020.json.gz,Apple cable conexión usb tipo c a usb-c 1m color blanco,"[apple, cable, conexión, usb, tipo, usb-c, 1m, color, blanco]"
8,17518365,323989,308,Product_gms-store.com_September2020.json.gz,Cabo USB-C para USB-C 1 metro,"[cabo, usb-c, usb-c, 1, metro]"
9,17912158,323989,5743,Product_euro.com.pl_September2020.json.gz,Apple Kabel USB-C,"[apple, kabel, usb-c]"


In [53]:
df_cluster_323989['jaccard_score'] = df_cluster_323989['product_tokens'].apply(lambda row: jaccard_similarity_score(row,df_cluster_323989.iloc[0]['product_tokens']) )

In [54]:
df_cluster_323989

,index,cluster_id,row_id,table_id,name,product_tokens,jaccard_score
0,18006,323989,192,Product_macs-4-u.com.au_September2020.json.gz,USB-C Charge Cable (1 m),"[usb-c, charge, cable, 1]",1.000000
1,1318608,323989,1064,Product_retravision.com.au_September2020.json.gz,Apple USB-C Charge Cable (1M),"[apple, usb-c, charge, cable, 1m]",0.500000
2,2826897,323989,152,Product_pskmegastore.com_September2020.json.gz,Apple MUF72ZM/A cavo USB 1 m USB C Bianco,"[apple, muf72zm/a, cavo, usb, 1, usb, bianco]",0.111111
3,3207585,323989,9,Product_ilovecomputers.com.au_September2020.json.gz,Apple Apple USB-C Charge Cable (1 m),"[apple, apple, usb-c, charge, cable, 1]",0.800000
4,12580514,323989,146,Product_mcarerental.com_September2020.json.gz,Apple USB-C Charge Cable 1m,"[apple, usb-c, charge, cable, 1m]",0.500000
5,15121729,323989,321,Product_alza.co.uk_September2020.json.gz,Apple USB-C Charging Cable 1m,"[apple, usb-c, charging, cable, 1m]",0.285714
6,15290618,323989,20,Product_lecbroome.com.au_September2020.json.gz,USB-C Charge Cable (2M),"[usb-c, charge, cable, 2m]",0.600000
7,16279424,323989,8,Product_expotronica.com_September2020.json.gz,Apple cable conexión usb tipo c a usb-c 1m color blanco,"[apple, cable, conexión, usb, tipo, usb-c, 1m, color, blanco]",0.181818
8,17518365,323989,308,Product_gms-store.com_September2020.json.gz,Cabo USB-C para USB-C 1 metro,"[cabo, usb-c, usb-c, 1, metro]",0.333333
9,17912158,323989,5743,Product_euro.com.pl_September2020.json.gz,Apple Kabel USB-C,"[apple, kabel, usb-c]",0.166667
